In [1]:
# The algorithm of Skew-Takagi factorization

using LinearAlgebra;

function j_matrix(n)
    if iseven(n)
        kron(Matrix{Int}(I, ÷(n,2), ÷(n,2)), [0 1; -1 0])
    else
        hvcat((2,2),kron(Matrix{Int}(I, ÷(n - 1,2), ÷(n - 1,2)), [0 1; -1 0]), zeros(Int8, 2*÷(n - 1,2), 1),zeros(Int8, 1, 2*÷(n - 1,2)),[1])
    end
end

function skew_Takagi(A)
    U, λ, V = svd(A)
    Diagonal(λ), U*sqrt(U'*conj.(V)*j_matrix(size(A,1)))
end
;

In [2]:
#Testing the algorithm
random_skew_sym(n) = ((x -> x - transpose(x))(rand(ComplexF64, (n, n))))

# Single- run testing
(function(n)
    A = random_skew_sym(n)
    Λ, U = skew_Takagi(A)

    print("Skewsymmetric (accuracy): ")
    println(norm(A + transpose(A)) )

    print("Unitarity (accuracy): ")
    println(norm(U * U' - I))

    print("Accuracy of skew-Takagi: ")
    println(norm(A - U*Λ*j_matrix(n)*transpose(U)))
end)(10)

Skewsymmetric (accuracy): 0.0
Unitarity (accuracy): 9.39813687563624e-15
Accuracy of skew-Takagi: 1.205079898875161e-14


In [3]:
#Timing
[(@timed skew_Takagi(random_skew_sym(i));).time for i=500:100:2000]

16-element Vector{Float64}:
  1.5159129
  2.5099436
  3.2421817
  4.5777299
  5.7942854
  7.3033936
  9.0856527
 11.9604611
 14.7685287
 18.3604238
 23.2231977
 28.3135898
 33.5860001
 40.6307958
 47.9096606
 55.4508337

In [4]:
# Computational error
function random_error_test(n)
    A = random_skew_sym(n)
    Λ, U = skew_Takagi(A)
    norm(A - U*Λ*j_matrix(n)*transpose(U))
end

[random_error_test(i) for i=500:100:2000]

16-element Vector{Float64}:
 7.55283465278524e-12
 8.997417491362346e-12
 1.1108542142774223e-11
 2.734233119687516e-11
 1.5593673559130724e-11
 1.9082330303034032e-11
 1.9061126544420372e-11
 2.1883333525282315e-11
 4.1461988328713364e-11
 2.548387461514827e-11
 2.7516437285272253e-11
 4.088673253499958e-11
 4.034601754800426e-11
 2.991751570159375e-10
 3.634964364994246e-11
 8.87667074272121e-11

In [5]:
# Application to quadratic Hamiltonians 

# Implementation of fermionic creation  - annihilation operators and quadratic Hamiltonians

σ3 = [1  0
      0 -1];
σp = [0 1
      0 0];
σm = [0 0
      1 0];

function cm(n, k)
    if (k==1)
        kron(σm, Matrix{Int}(I, 2^(n-1), 2^(n-1)))
    elseif (1<k<n)
        kron(kron(reduce(kron,repeat([σ3],k-1)),σm), Matrix{Int}(I, 2^(n-k), 2^(n-k)))
    else
        kron(reduce(kron,repeat([σ3],n-1)),σm)
    end
end

function cp(n, k)
    if (k==1)
        kron(σp, Matrix{Int}(I, 2^(n-1), 2^(n-1)))
    elseif (1<k<n)
        kron(kron(reduce(kron,repeat([σ3],k-1)),σp), Matrix{Int}(I, 2^(n-k), 2^(n-k)))
    else
        kron(reduce(kron,repeat([σ3],n-1)),σp)
    end
end

function Ham(A)
    n = size(A,1);
    sum((A[i,j]*cp(n, i)*cp(n, j) + conj(A[i,j])*cm(n, i)*cm(n, j)) for  i=1:n, j=1:n) * 0.5im
end
;

In [6]:
# Algorithms for Heisenberg evolution based on skew-Takagi factorization and on direct computaion of matrix exponential
function ct_Heisenberg(A, t)
    n = size(A,1);
    Ut = exp(1.0im*Ham(A));
    [(Ut * cm(n, k) * Ut') for k=1:n]
end

function skew_Takagi_based_ct_Heisenberg(A, t)
    Λ, U = skew_Takagi(A);
    n = size(A,1);
    [ sum((U * cos(Λ*t) * U')[i, j] * cm(n, j) + (U * sin(Λ*t) * j_matrix(n) * transpose(U))[i, j] * cp(n, j) for j=1:n) for i=1:n]
end
;

In [7]:
# Test that both methods coincide
(function(n,t)
    A = random_skew_sym(n);
    map(norm, ct_Heisenberg(A, t)-skew_Takagi_based_ct_Heisenberg(A, t))
end)(4,1)

4-element Vector{Float64}:
 2.06522861658258e-15
 3.1030003650644603e-15
 3.855959952432236e-15
 4.122462465694189e-15

In [8]:
# Timing comparison
function timing_comparison(n)
    t=1;
    A = random_skew_sym(n);
    [n (@timed ct_Heisenberg(A, t);).time (@timed skew_Takagi_based_ct_Heisenberg(A, t);).time]
end

[timing_comparison(n) for n=1:10]

10-element Vector{Matrix{Float64}}:
 [1.0 0.8351782 0.358471]
 [2.0 0.0001506 4.66e-5]
 [3.0 0.0004029 0.0001374]
 [4.0 0.0010698 0.0001967]
 [5.0 0.005034 0.0015424]
 [6.0 0.0893884 0.0071683]
 [7.0 0.3084742 0.0439465]
 [8.0 2.2697802 0.1435564]
 [9.0 22.4252152 0.7591133]
 [10.0 226.5407074 3.0079538]